# Assignment 8

#### 1. 复习上课内容

#### 2.理论题

##### 1.  What is semi-supervised learning ?

半监督学习是模式识别和机器学习领域研究的重点问题，是监督学习与无监督学习相结合的一种学习方法。半监督学习使用大量的未标记数据，以及同时使用标记数据，来进行模式识别工作。
半监督学习主要包括生成式方法、半监督SVM、图半监督学习、基于分歧的方法。

##### 2. What is GMM ?

高斯混合模型的思想是将数据集中的点（通常是二维数据点）视为多个正态分布，也就是高斯函数，互相线性叠加的结果。通过估计各个高斯函数的中心位置和标准差，拟合数据集中的点分布，进而确定各个点对应的高斯函数中心，划归于对应的类。

##### 3. What are the diffreences between seed k-means and constrained k-means ?

Seeded K-means用已标注数据初始化各个聚类的中心，以后的过程中，已标注数据的标签信息可以改变。

Constrained K-means已标注数据初始化各个聚类的中心，以后的过程中，已标注数据的标签信息不得改变。

##### 4. Please briefly describe what are self-training and co-training ? 

自训练（self-training）利用已标记的样本训练分类器，然后利用已有未标记训练样本建立的模型标记，使用分类器选出置信度高的样本加入训练集中重新训练，迭代这个过程，得到一个比较理想的分类器。

适用：用监督学习分类器时很复杂或者是很难修正

优点：简单、容易实现。

缺点：误差也同时会自我迭代放大。

协同训练（co-training）假设特征能够分成两个集，这两个数据集是独立同分布的。每个子特征集能够训练一个很好的分类器。每个分类器把未标记的样本分出来给另一个分类器，选择出置信度高的新的为样本进行标记，重复上面的过程。

适用：特征能够很好的分成两类。

缺点：大多数的问题并不具有“充分大”的属性集，而且随机划分视图这一策略并非总能奏效。

##### 5. What is active learning ?

主动学习方法假设学习器对系统的状态就有一定的“控制能力”，可以主动地按照规则选择一些未标记的样本，并通过用户交互或者是能够为样本提供真实标记的其他过程，得到这些样本的标记，然后再将这些有标记的样本作为训练集的一部分，融入到系统的学习器中进行监督式学习。是通过对特定样本的采样来引导分类器的训练过程的学习方式。

机器学习一般是被动学习，根据数据进行学习。主动学习根据模型挑数据进行学习。

#### 3.实践题

Titanic: Machine Learning from Disaster

Data Source :https://www.kaggle.com/c/titanic/data

####  In this assignment, you have to build a predictive model that answers the question: “what sorts of people were more likely to survive?” using passenger data (ie name, age, gender, socio-economic class, etc).

低： 完成。

高：准确率 95% 以上。

In [72]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt
%matplotlib inline

In [38]:
train = pd.read_csv('../titanic/train.csv')
test = pd.read_csv('../titanic/test.csv')

In [40]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [41]:
X_full = pd.concat([train.drop('Survived', axis = 1), test], axis = 0) #concat函数是在pandas底下的方法，可以将数据根据不同的轴作简单的融合

In [42]:
X_full.shape

(1309, 11)

In [43]:
X_full.drop('PassengerId',axis=1,inplace=True)

In [44]:
X_full.isnull().sum()

Pclass         0
Name           0
Sex            0
Age          263
SibSp          0
Parch          0
Ticket         0
Fare           1
Cabin       1014
Embarked       2
dtype: int64

In [45]:
(X_full.Age.isnull() & X_full.Cabin.isnull()).sum() 

240

In [46]:
train.Survived.mean()

0.3838383838383838

In [47]:
train.Cabin.notnull().mean()

0.22895622895622897

In [48]:
(train.Cabin.isnull() & (train.Survived==0)).mean()

0.5398428731762065

In [49]:
selector = (train.Cabin.isnull() & train.Age.isnull())
train[selector].Survived.mean()

0.25949367088607594

In [50]:
X_full['Nulls'] = X_full.Cabin.isnull().astype('int') + X_full.Age.isnull().astype('int')

In [51]:
X_full['Cabin_mapped'] = X_full['Cabin'].astype(str).str[0]
cabin_dict = {k:i for i, k in enumerate(X_full.Cabin_mapped.unique())}
X_full.loc[:, 'Cabin_mapped'] = X_full.loc[:, 'Cabin_mapped'].map(cabin_dict)

In [52]:
cabin_dict

{'n': 0, 'C': 1, 'E': 2, 'G': 3, 'D': 4, 'A': 5, 'B': 6, 'F': 7, 'T': 8}

In [53]:
X_full.drop(['Age','Cabin'], inplace=True, axis=1)

In [54]:
fare_mean = X_full[X_full.Pclass == 3].Fare.mean()

X_full['Fare'].fillna(fare_mean,inplace = True)

In [55]:
X_full.isnull().sum()

Pclass          0
Name            0
Sex             0
SibSp           0
Parch           0
Ticket          0
Fare            0
Embarked        2
Nulls           0
Cabin_mapped    0
dtype: int64

In [56]:
X_full[X_full.Embarked.isnull()]

,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Embarked,Nulls,Cabin_mapped
61,1,"Icard, Miss. Amelie",female,0,0,113572,80.0,NaN,0,6
829,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,0,0,113572,80.0,NaN,0,6


In [57]:
X_full[X_full['Pclass'] == 1].Embarked.value_counts()

S    177
C    141
Q      3
Name: Embarked, dtype: int64

In [58]:
X_full['Embarked'].fillna('S', inplace = True)

In [59]:
X_full.isnull().sum()

Pclass          0
Name            0
Sex             0
SibSp           0
Parch           0
Ticket          0
Fare            0
Embarked        0
Nulls           0
Cabin_mapped    0
dtype: int64

In [60]:
X_full.drop(['Name', 'Ticket'], axis = 1, inplace = True)

In [62]:
X_full.dtypes

Pclass            int64
Sex              object
SibSp             int64
Parch             int64
Fare            float64
Embarked         object
Nulls             int32
Cabin_mapped      int64
dtype: object

In [63]:
X_dummies = pd.get_dummies(X_full, columns = ['Sex', 'Nulls', 'Cabin_mapped', 'Embarked'], drop_first= True)

In [64]:
X_dummies.dtypes

Pclass              int64
SibSp               int64
Parch               int64
Fare              float64
Sex_male            uint8
Nulls_1             uint8
Nulls_2             uint8
Cabin_mapped_1      uint8
Cabin_mapped_2      uint8
Cabin_mapped_3      uint8
Cabin_mapped_4      uint8
Cabin_mapped_5      uint8
Cabin_mapped_6      uint8
Cabin_mapped_7      uint8
Cabin_mapped_8      uint8
Embarked_Q          uint8
Embarked_S          uint8
dtype: object

In [65]:
X = X_dummies[:len(train)]; new_X = X_dummies[len(train):]
y = train.Survived

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state = 5,
                                                    stratify = y)

## RandomForest

In [73]:
rf = RandomForestClassifier()

rf.fit(X_train, y_train)

rf.score(X_test, y_test)

0.7910447761194029

## XGBOOST

In [74]:
xgb = XGBClassifier()

xgb.fit(X_train, y_train)

xgb.score(X_test, y_test)

0.8097014925373134

## LogisticRegression

In [76]:
lg = LogisticRegression()

lg.fit(X_train, y_train)

lg.score(X_test, y_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8022388059701493

In [ ]:
#有待提高